<a href="https://colab.research.google.com/github/Priyo-prog/Artificial-Intelligence-Projects/blob/main/Healthcare%20AI/healthcare_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Healthcare AI**

* AI is revolutionizing Healthcare in many areas such as :
  * Disease Diagonosis with medical imaging
  * Surgical Robots
  * Maximizing Hospital Efficiency

* Deep learning has been proven to be superior in detecting disease from X-rays, MRI scans and CT scans which could significantly improve the speed and accuracy of diagnosis.   

AIM :
* You have been tasked to improve the speed and accuracy of detecting and localizing brain tumors based on MRI scans.
* This would drastically reduce the cost of cancer diagnosis and help in early diagnosis of tumors which would essentially be a life saver.

* The team has collected brain MRI scans and approached you to develop a model that could detect and localize a tumor.

* You have been provided with 3929 brain MRI scans along with their brain tumor location.



**WHAT IS IMAGE SEGMENTATION ?**

* The goal of image segmentation is to understand and extract information from images at pixel-level.
* Image segmentation can be used for object recognition and localization which offers tremendous value in many applications such as medical imaging and self-driving cars etc.
* The goal of image segmentation is to train a neural network to produce pixel-wise mask of the image.
* Modern image segmentation techniques are based on deep learning approcah which makes use of common architectures such as CNN, FCNs(Fully Convolutional Networks) and Deep Encoders-Decoders.
* You will use ResUNet architecture to solve the current task.

## **Import Libraries and Packages**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
import os
import glob
import random
from google.colab import files # library to upload files to colab notebook
%matplotlib inline

## **Mount Drive and Make Default Folder For Uploading Dataset**

In [3]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Navigate to My Drive directory to store dataset
%cd /content/drive/MyDrive/Artificial Intelligence Projects/Healthcare AI/Brain MRI

/content/drive/MyDrive/Artificial Intelligence Projects/Healthcare AI/Brain MRI


In [6]:
# data containing path to brain MRI and their corresponding mask
brain_df = pd.read_csv("data_mask.csv")

In [7]:
brain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3929 entries, 0 to 3928
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  3929 non-null   object
 1   image_path  3929 non-null   object
 2   mask_path   3929 non-null   object
 3   mask        3929 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 122.9+ KB


In [8]:
brain_df.head(5)

,patient_id,image_path,mask_path,mask
0,TCGA_CS_5395_19981004,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1.tif,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1_...,0
1,TCGA_CS_5395_19981004,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1.tif,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1_...,0
2,TCGA_CS_5395_19981004,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1.tif,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1_...,0
3,TCGA_CS_5395_19981004,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1.tif,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1_...,0
4,TCGA_CS_5395_19981004,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1.tif,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1_...,0


**MASK**

* The goal of image segmentation is to understand the image at pixel level. It associate each pixel with a certain class. The output produced by image segmentation model is called a "mask" of the image.

* Mask can be represented by associating pixel values with their coordinates. For example if we have a black image of shape (2,2) this can be represented as:

    [[0,0]

    [0,0]]

* If our output mask is as follows:

  [[255,0]

   [0,255]]

* To represent this mask we have to first flatten the image into a 1-D array. This would result in something like [255,0,0,255] for mask. Then we can use the index to create the mask. Finally we would have something like [1,0,0,1] as our mask.   

In [10]:
# Find whether the dataset is balanced or not
brain_df["mask"].value_counts()

0    2556
1    1373
Name: mask, dtype: int64

## **Perform Data Visualization**

In [12]:
brain_df["mask"].value_counts().index

Int64Index([0, 1], dtype='int64')

In [14]:
# Use plotly to plot interactive bar chart
import plotly.graph_objects as go

fig = go.Figure([go.Bar(x = brain_df["mask"].value_counts().index , y = brain_df["mask"].value_counts())])
fig.update_traces(marker_color = "rgb(0,200,0)", marker_line_color = "rgb(255,0,0)",
                        marker_line_width = 3, opacity = 0.6)
      
fig.show()      